In [1]:
import gym
import tensorflow as tf
import numpy as np
import collections
import random

In [2]:
env = gym.make('CartPole-v1')
actsize = env.action_space.n
stateSize = env.observation_space.shape[0]
sess = tf.Session()

In [3]:
class network(object):
    def __init__(self,session = None,input_dim = None,output_dim = None,lr = None):
        self.inputDim = input_dim
        self.outputDim = output_dim
        self.learningRate = lr
        #构建深度训练网络
        self.__buildNN()
        self.sess = session
        self.sess.run(tf.global_variables_initializer())
        self.sess.run(tf.local_variables_initializer())
        
    def __buildNN(self):
        w_initializer, b_initializer = tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)
        self.X = tf.placeholder(dtype=tf.float32,shape=(None,self.inputDim))
        self.Q_traget = tf.placeholder(dtype=tf.float32, shape=(None,self.outputDim), name='Q_target')
        h1 = tf.layers.dense(inputs=self.X, units=16, activation=tf.nn.relu,kernel_initializer=w_initializer,bias_initializer=b_initializer)
        h2 = tf.layers.dense(inputs=h1, units=16, activation=tf.nn.relu,kernel_initializer=w_initializer,bias_initializer=b_initializer)
        self.Y = tf.layers.dense(inputs=h1, units=self.outputDim,kernel_initializer=w_initializer,bias_initializer=b_initializer)
        self.loss = tf.reduce_mean(tf.squared_difference(self.Q_traget,self.Y))  #TD-ERROR loss
        self._train_op = tf.train.RMSPropOptimizer(self.learningRate).minimize(self.loss)    

    def train(self,qvalue = None, predict = None):
        _, cost = self.sess.run([self._train_op,self.loss],feed_dict={self.X:predict, self.Q_traget:qvalue})
        return cost
        
    def predict(self,Input = None):
        evalData = self.sess.run(self.Y,feed_dict={self.X:Input})
        return evalData
        

In [4]:
nn = network(sess,input_dim = stateSize, output_dim = actsize,lr = 0.01)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [5]:
class dqn(object):
    def __init__(self,statesize = None,actionsize = None,env = None,network= None,gamma = 0.9,epsilon = 0.5,replaySize = 2000,batchsize = 32):
        self._statesize = statesize
        self._actionsize = actionsize
        self._epsilon_min = 0.01
        self._nn = network
        self._gamma = gamma
        self._epsilon = epsilon
        self._env = env
        self._replay = collections.deque(maxlen=replaySize)
        self._bacthsize = batchsize
    
    def rember(self,s,s_,a,r,done):
        if len(self._replay) == self._replay.maxlen:
            self._replay.popleft()
        self._replay.append((s,s_,a,r,done))
        
    def learning(self):
        cost = 0
        if len(self._replay) > self._bacthsize:
            #随机抽取记忆数据进行训练
            minibatch = random.sample(self._replay, self._bacthsize)
            qvalues = []
            states = []
            for s,s_,a,r,done in minibatch:
                states.append(s)
                if not done:
                    value = self._nn.predict(s_)
                    idx = sess.run(tf.math.argmax(value,1))
                    max_action = value[0][idx[0]]
                    qtarget = r + self._gamma * max_action
                else:
                    qtarget = reward
                newvalue = self._nn.predict(s)
                newvalue[0][a] = qtarget
                qvalues.append(newvalue)
            qvalues = np.array(qvalues).reshape(-1,2)
            states = np.array(states).reshape(-1,4)
            cost = self._nn.train(qvalues,states)
            if self._epsilon > self._epsilon_min:
                self._epsilon -= (0.5 - self._epsilon_min)/10000
        return cost
    
    def greedy_act(self,state):
        if np.random.rand() <= self._epsilon:
            return random.randrange(self._actionsize)
        pY = self._nn.predict(state)
        idx = sess.run(tf.math.argmax(pY,1))
        return idx[0]
        

In [7]:
DQN = dqn(stateSize,actsize,env,nn)
# pY = nn.predict(state)
# idx = sess.run(tf.math.argmax(pY[0],1))
for i in range(500):
    state = env.reset()
    state = np.array(state).reshape(1,4)
    for j in range(200):
        action = DQN.greedy_act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.array(next_state).reshape(1,4)
        DQN.rember(state,next_state,action,reward,done)
        if done:
            print("Failed:{} at {}/500".format(j,i))
            break
        state = next_state
    cost = DQN.learning()
    
    

Failed:25 at 0/500
Failed:29 at 1/500
Failed:20 at 2/500
Failed:14 at 3/500
Failed:22 at 4/500
Failed:10 at 5/500
Failed:10 at 6/500
Failed:34 at 7/500
Failed:57 at 8/500
Failed:31 at 9/500
Failed:37 at 10/500
Failed:24 at 11/500
Failed:68 at 12/500
Failed:43 at 13/500
Failed:33 at 14/500
Failed:51 at 15/500
Failed:24 at 16/500
Failed:22 at 17/500
Failed:42 at 18/500
Failed:65 at 19/500
Failed:18 at 20/500
Failed:33 at 21/500
Failed:21 at 22/500
Failed:10 at 23/500
Failed:14 at 24/500
Failed:9 at 25/500
Failed:14 at 26/500
Failed:20 at 27/500
Failed:17 at 28/500
Failed:47 at 29/500
Failed:61 at 30/500
Failed:62 at 31/500
Failed:50 at 32/500
Failed:51 at 33/500
Failed:44 at 34/500
Failed:30 at 35/500
Failed:26 at 36/500
Failed:14 at 37/500
Failed:15 at 38/500
Failed:36 at 39/500
Failed:21 at 40/500
Failed:21 at 41/500
Failed:26 at 42/500
Failed:28 at 43/500
Failed:46 at 44/500
Failed:18 at 45/500
Failed:28 at 46/500
Failed:20 at 47/500
Failed:27 at 48/500
Failed:14 at 49/500
Failed:27 a

AttributeError: 'dqn' object has no attribute 'memory'

In [13]:
print(tf.GraphKeys.LOCAL_VARIABLES)

local_variables
